In [3]:
from sklearn.datasets import fetch_20newsgroups
# LDA는 빈도수에만 기반하는 CountVectorizer 사용함!
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [8]:
# 주어진 데이터셋의 일부 카테고리 데이터만 추출하므로 카테고리 사전에 설정
cats = ['rec.motorcycles', 'rec.sport.baseball', 'comp.graphics', 'comp.windows.x',
        'talk.politics.mideast', 'soc.religion.christian', 'sci.electronics', 'sci.med']

# 설정해준 카테고리의 데이터들만 추출
news_df = fetch_20newsgroups(subset = 'all', remove = ('headers', 'footers', 'quotes'),
                             categories = cats, random_state = 12)

In [10]:
# CountVectorizer로 텍스트 데이터를 단어 빈도수에 기반해 벡터화시키기 (fit_transform까지)
count_vect = CountVectorizer(max_df = 0.95, max_features = 1000,
                             min_df = 2, stop_words = 'english',
                             ngram_range = (1, 2))
fit_vect = count_vect.fit_transform(news_df.data)

In [11]:
# LDA 클래스를 이용해서 피쳐 벡터화시킨 것을 토픽 모델링 시키기
# 8개의 주제만 뽑았으니 n_components 8로 설정
lda = LatentDirichletAllocation(n_components = 8, random_state =42)
lda.fit(fit_vect)

LatentDirichletAllocation(n_components=8, random_state=42)

In [13]:
# components_ 속성은 8개의 토픽별(row)로 1000개의 feature(단어)들의 분포수치(column)를 보여줌
print(lda.components_.shape)
print(lda.components_)

# 행은 각각의 토픽, 열은 단어들을 벡터화시킨 features

(8, 1000)
[[4.21886779e+01 1.25077441e-01 1.03902744e+01 ... 3.28675046e+01
  1.25032926e-01 2.23626159e+00]
 [3.31601110e+02 1.13269256e+02 1.62846341e+02 ... 1.38787171e-01
  2.40125634e+02 3.98253576e+00]
 [1.27625269e-01 9.96794709e+01 1.25480119e-01 ... 7.24201742e+00
  1.90971768e+01 5.07321349e+01]
 ...
 [1.25089768e-01 3.89851204e+01 1.25038018e-01 ... 2.14401784e+02
  1.25058173e-01 9.50015891e+01]
 [1.25041718e-01 2.43565827e+02 1.25013778e-01 ... 2.25580117e+01
  2.46882968e+01 4.56696629e+01]
 [1.25067731e-01 1.25058566e-01 1.25000680e-01 ... 1.20416570e+02
  1.25047652e-01 3.96898176e+01]]


In [14]:
# 토픽 별로 어떤 단어들이 많이 분포하는지 시각적으로 보기 위한 함수
# 이 때 lda_model이란, 벡터화시킨 텍스트 데이터를 fit까지만 적용한 모델
def display_topic_words(ida_model, features_names, num_top_words):
    for topic_idx, topic in enumerate(ida_model.components_):
        print('\nTopic #', topic_idx + 1)

        # Topic 별로 1000개의 단어들(features) 중에서 높은 값 순으로 정렬 후 index를 반환해줌
        # argsort()는 디폴트가 오름차순임. 그래서 [::-1]로 내림차순으로 바꿔주기
        topic_word_idx = topic.argsort()[::-1]
        top_idx = topic_word_idx[:num_top_words]

        # CountVectorizer 함수 할당시킨 객체에 get_feature_names()로 벡터화시킨 feature 볼 수 있음
        # 이 벡터화시킨 단어들(features)은 숫자-알파벳 순으로 정렬되며, 단어들 순서는 fit_transform 시키고 난 이후에도 동일
        feature_concat = '+'.join([str(features_names[i]) + '*' + str(round(topic[i], 1)) for i in top_idx])
        print(feature_concat)

In [18]:
feature_names = count_vect.get_feature_names_out()
display_topic_words(lda, feature_names, 15)


Topic # 1
file*1159.4+use*991.4+jpeg*784.4+program*783.8+window*763.5+image*568.2+output*538.4+display*533.8+color*525.8+using*493.1+files*455.3+gif*432.8+bit*400.6+entry*396.9+set*383.0

Topic # 2
university*372.0+00*331.6+new*317.4+03*266.7+ed*257.8+02*254.4+04*246.6+york*240.1+new york*239.1+ground*205.6+adl*189.1+wire*188.0+circuit*186.1+professor*177.5+san*169.6

Topic # 3
israel*805.1+israeli*475.8+medical*437.0+10*426.1+research*384.2+health*376.2+1993*346.9+arab*339.3+disease*333.1+cancer*321.1+patients*303.1+12*292.1+use*274.7+number*274.3+april*268.8

Topic # 4
edu*1584.7+graphics*998.5+available*855.8+software*792.8+com*760.2+ftp*759.8+image*725.9+dos*684.1+data*605.6+version*556.8+pub*553.4+windows*529.2+mail*449.7+server*419.2+computer*418.3

Topic # 5
know*1067.3+like*857.8+thanks*788.2+does*671.1+just*624.7+don*559.2+bike*540.1+ve*473.8+help*447.9+mail*391.4+edu*389.6+need*383.6+want*371.1+looking*340.4+use*340.4

Topic # 6
don*1415.4+just*1240.6+like*1077.8+think*1067.

In [20]:
# transform 함수를 호출해서 '문서별(row) 토픽들(columns)의 분포'까지 살필 수 있음
doc_topics = lda.transform(fit_vect)
print(doc_topics.shape)
print(doc_topics[:2])

# 행은 각 문서들, 열은 각 토픽들

(7862, 8)
[[0.41998894 0.00543729 0.00543985 0.00544059 0.00544265 0.54737167
  0.00543803 0.00544097]
 [0.01564771 0.01564216 0.77994069 0.12615745 0.01565929 0.01563803
  0.01564152 0.01567315]]


In [21]:
import pandas as pd
# 주어진 내장 텍스트 데이터의 문서 이름에는 카테고리가 labeling되어 있음
# 따라서, 카테고리가 무엇인지 아는 상태이니까 어떤 문서들이 어떤 토픽들이 높은지 확인해보자
# 그리고 그 토픽들이 각각 무엇을 내용으로 하는지 추측해보자
# 주어진 데이터셋의 filename 속성을 이용해서 카테고리 값들 가져오기


def get_filename_list(newsdata):
    filename_lst = []
    for file in newsdata.filenames:
        filename_temp = file.split('/')[-2:]
        filename = '.'.join(filename_temp)
        filename_lst.append(filename)
    return filename_lst

filename_lst = get_filename_list(news_df)


In [22]:
# DataFrame 형태로 만들어보기
topic_names = ['Topic #' + str(i) for i in range(0,8)]
topic_df = pd.DataFrame(data = doc_topics, columns = topic_names, index = filename_lst)
print(topic_df.head(20))

                              Topic #0  Topic #1  Topic #2  Topic #3  \
comp.graphics.38765           0.419989  0.005437  0.005440  0.005441   
sci.med.59107                 0.015648  0.015642  0.779941  0.126157   
sci.electronics.54182         0.005444  0.005446  0.005440  0.005450   
rec.motorcycles.103182        0.007356  0.007360  0.007354  0.007355   
soc.religion.christian.21740  0.001739  0.001739  0.001738  0.001738   
rec.sport.baseball.105077     0.000568  0.996022  0.000568  0.000568   
talk.politics.mideast.75974   0.002274  0.002275  0.385195  0.002275   
talk.politics.mideast.76050   0.012513  0.012506  0.012522  0.230723   
soc.religion.christian.20900  0.015639  0.015626  0.015631  0.015629   
sci.electronics.54334         0.005693  0.005688  0.005696  0.005693   
rec.sport.baseball.102625     0.007816  0.007839  0.007833  0.007815   
sci.electronics.53888         0.013908  0.013893  0.013901  0.013910   
rec.motorcycles.104346        0.125000  0.125000  0.125000  0.12